## TO DO's

* Add yattag, markdown into base_notebook pip requirements
* Put this inside a grizly tool module (module name could be `mdy.py`)
* We need to link this to the make_html.py. We need to think of the right folder structure for docs and how we generate the docs and the sidebar from a combination of the folder structure and the headers in the files

## Concepts

MDY stands for Markdown + Yaml. Yaml blocks are included in `::block` and `::endblock` tags. Yaml blocks always have a **component** key. The component key is mapped to a python function that translate the yaml component into an html component

In [12]:
import re
import yaml
import markdown
from yattag import Doc, indent

def html_head():
    doc, tag, text = Doc().tagtext()
    doc.asis('<!DOCTYPE html>')
    with tag('html'):
        with tag('head'):
            with tag('link', rel="stylesheet", href="somecss.css"):
                text("")
    doc.asis('<body>')
    return doc.getvalue()

def html_footer():
    doc, tag, text = Doc().tagtext()
    doc.asis('</body>')
    return doc.getvalue()

def card_component(d):
    doc, tag, text = Doc().tagtext()
    with tag('div', klass="card"):
        with tag('h1'):
            text(d["title"])
        with tag("h2"):
            text(d["subtitle"])
    return doc.getvalue()

class MDY():
    def __init__(self, txt_path):
        self.txt_path = txt_path
        with open(txt_path) as f:
            self.txt = f.read()
        self.html = ""
        self.headers = {}
        self.regex_str = "::block(.*?)::endblock"
    
    def create_component(self, yaml_txt):
        d = yaml.safe_load(yaml_txt)
        try:
            component = d["component"]
            if component == "card":
                html = card_component(d)
            return html
        except KeyError:
            return "Error: component key missing"
    
    def yaml_to_html(self):
        pattern = re.compile(self.regex_str , re.DOTALL)
        matches = pattern.findall(self.txt)
        for match in matches:
            html = self.create_component(match)
            self.txt = self.txt.replace(match, html)
        return self
    
    def to_html(self):
        self.yaml_to_html()
        self.html = indent(html_head() + markdown.markdown(self.txt) + html_footer())
        self.html = self.html.replace("::block","")
        self.html = self.html.replace("::endblock","")
        return self
    
    def create_toc(self):
        """Creating elements for TOC (Table of Contents)"""

        with open(self.txt_path) as f:
            txt = f.read()
            pattern = re.compile(r"(?<=^# )(.*)", re.MULTILINE)
            h1 = pattern.findall(str(txt))
            pattern = re.compile(r"(?<=^## )(.*)",  re.MULTILINE)
            h2 = pattern.findall(str(txt))
            pattern = re.compile(r"(?<=^### )(.*)",  re.MULTILINE)
            h3 = pattern.findall(str(txt))
        self.headers["h1"] = h1
        self.headers["h2"] = h2
        self.headers["h3"] = h3
        return self
    
    

MDY("input.txt").create_toc().headers

{'h1': ['h1', 'some header with h1'], 'h2': ['h2', 'h2'], 'h3': ['h3', 'h3']}

### Creating HTML

First of all we can create html easily with `to_html()`

In [17]:
html = MDY("/home/analyst/grizly/notebooks/mkd_to_html.txt").to_html()
print(html.html)

<!DOCTYPE html>
<html>
  <head>
    <link rel="stylesheet" href="somecss.css"></link>
  </head>
</html>
<body>
  <h1>h1</h1>
  <h2>h2</h2>
  <h3>h3</h3>
  <p><div class="card"><h1>a simple title</h1><h2>a simple subtitle</h2></div></p>
  <h1>some header with h1</h1>
  <h2>h2</h2>
  <h3>h3</h3>
  <p>Error: component key missing</p>
  <p>etc we do not know exactly what the meaning of life should be but we do know that etc. etc. etc. what ever else</p>
  <p>well what else
etc</p>
</body>


### Creating the TOC (Table of Contents)

This is going to be useful to automate the creation of the sidebars for docs. We can develop a `TOC` based on the h1, h2 and h3 tags coming from the corresponding mardown

* \# -> H1
* \## -> H2
* \### -> H3


In [19]:
MDY("/home/analyst/grizly/notebooks/mkd_to_html.txt").create_toc().headers

{'h1': ['h1', 'some header with h1'], 'h2': ['h2', 'h2'], 'h3': ['h3', 'h3']}